# **Data Mining I | <span style="color: darkseagreen;">Exploratory Data Analysis and Pre-Processing</span>**

## A Clustering-Based Exploration of WorkplaceAbsence Patterns

*Group 07*

*Group members* : Francisco Gomes (20221810), Margarida Marchão, Pedro Coimbras and Marta Alves.

<a id="1"></a>

# **1. Importing the datasets and libraries <font>**

In [2]:
#imports
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pylab import rcParams

from utils1 import (
    get_education_level_and_clean_name,
    calculate_percentage,
    customized_histograms,
    customized_bar_charts,
    customized_pie_charts,
    customized_scatter,
    filled_line_plot,
    plot_lifetime_spending,
    plot_correlation_matrix,
    plot_k_distance,
    detect_outliers_dbscan,
    visualize_outliers_vs_normal,
    high_correlations
)
pd.set_option("display.max_columns", None)

In [3]:
#from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# path = '/content/drive/MyDrive/Colab Notebooks/DM1/projeto/data/'
path2 = '/Users/franciscogomes/Desktop/Faculdade/Master/Data Mining/Project/reasons_for_absenteeism/data/'

<a id="1"></a>

# **2. Preliminary Data Analysis<font>**

In [12]:
pd.set_option("display.max_columns", None)
abs = pd.read_csv(path2 + 'absenteeism_data.csv',delimiter=';')
abs = abs.set_index("ID")
abs.head(0)


,Reason for absence,Month of absence,Day of the week,Seasons,Days since previous absence,Transportation expense,Distance from Residence to Work,Estimated commute time,Service time,Years until retirement,Date of Birth,Disciplinary failure,Education,Number of children,Social drinker,Social smoker,Number of pets,Weight,Height,Body mass index,Absenteeism time in hours
ID,,,,,,,,,,,,,,,,,,,,,


okay, lets gOOooOOoOOo

In [5]:
display(abs.shape)

(800, 21)

<a id="1"></a>

### **2.1. Data Types <font>**

In [6]:
display(abs.info())

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 11 to 35
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Reason for absence               740 non-null    object 
 1   Month of absence                 737 non-null    object 
 2   Day of the week                  740 non-null    object 
 3   Seasons                          573 non-null    object 
 4   Days since previous absence      781 non-null    float64
 5   Transportation expense           800 non-null    int64  
 6   Distance from Residence to Work  800 non-null    int64  
 7   Estimated commute time           800 non-null    int64  
 8   Service time                     800 non-null    object 
 9   Years until retirement           800 non-null    int64  
 10  Date of Birth                    800 non-null    object 
 11  Disciplinary failure             800 non-null    object 
 12  Education                  

None

 - missing values in: reason of absence, month of absence, day of the week, seasons, days since previous absence
 - education is int
 - we can predict seasons with month of absence
 - transportation expense is in brasilian reais -> mudar para euros
 - service time and weight has missing values '-' -> mudar para int
 - one hot no disciplinary failure, social frinker and smoker
 - social drinker and smoker has values of 'Y'
 - ver 0s na coluna do absenteeism time in hours -> 0h can mean 30min or it wasnt absent
 - maybe mudar int64 para int32
 - dar drop do days since previous absence porque só tem um unico valor que é 0.0
 - ver outliers
 - compute age
 - há duplicates - 48

<a id="1"></a>

### **2.4. Missing Values <font>**

In [7]:
abs.isna().sum()

Reason for absence                  60
Month of absence                    63
Day of the week                     60
Seasons                            227
Days since previous absence         19
Transportation expense               0
Distance from Residence to Work      0
Estimated commute time               0
Service time                         0
Years until retirement               0
Date of Birth                        0
Disciplinary failure                 0
Education                            0
Number of children                   0
Social drinker                       0
Social smoker                        0
Number of pets                       0
Weight                               0
Height                               0
Body mass index                      0
Absenteeism time in hours            0
dtype: int64

In [8]:
missing_columns = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Days since previous absence']
missing_percentages = calculate_percentage(abs, columns=missing_columns)

for col, percent in missing_percentages.items():
    display(f'Around {percent}% of the data points in the column {col} are missing values')

'Around 7.5% of the data points in the column Reason for absence are missing values'

'Around 7.9% of the data points in the column Month of absence are missing values'

'Around 7.5% of the data points in the column Day of the week are missing values'

'Around 28.4% of the data points in the column Seasons are missing values'

'Around 2.4% of the data points in the column Days since previous absence are missing values'

estamos a ver o porque de algumas variaves serem object ------------------------------------------------------------------------------

<a id="1"></a>

### **2.5. Summary Statistics - (numerical data) <font>**

In [9]:
abs.describe().T

,count,mean,std,min,25%,50%,75%,max
Days since previous absence,781.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Transportation expense,800.0,221.92750,66.778732,118.0,179.0,225.0,260.0,388.0
Distance from Residence to Work,800.0,29.79875,14.875057,5.0,16.0,26.0,50.0,52.0
Estimated commute time,800.0,59.34875,31.301067,5.0,31.0,52.0,94.0,114.0
Years until retirement,800.0,26.74250,9.269498,-1.0,25.0,28.0,33.0,38.0
Education,800.0,1.28750,0.667383,1.0,1.0,1.0,1.0,4.0
Number of children,800.0,1.04375,1.120672,0.0,0.0,1.0,2.0,4.0
Number of pets,800.0,0.74500,1.323694,0.0,0.0,0.0,1.0,8.0
Height,800.0,172.09375,6.058122,163.0,169.0,170.0,172.0,196.0
Body mass index,800.0,26.64125,4.267354,19.0,24.0,25.0,31.0,38.0


Days since previous absence seems weird because the variable only contains 0 or NaN

In [18]:
display(abs['Service time'].unique())

array(['13', '18', '14', '3', '11', '16', '4', '6', '12', '7', '10', '-',
       '9', '17', '29', '8', '1', '15', '24'], dtype=object)

In [19]:
display(abs['Date of Birth'].unique())

array(['1992-08-15', '1975-09-02', '1987-04-08', '1986-07-25',
       '1997-07-08', '1989-02-20', '1991-09-10', '1988-06-01',
       '1984-10-01', '1992-05-24', '1978-02-16', '1997-01-19',
       '1996-05-22', '1977-04-28', '1993-04-25', '1998-12-19',
       '1988-05-10', '1982-06-19', '1985-05-20', '1997-01-01',
       '1997-08-03', '1994-06-26', '1995-08-01', '1985-06-16',
       '1975-06-07', '1989-02-10', '1976-09-21', '1967-11-02',
       '1982-08-04', '1992-04-20', '1986-09-13', '1993-11-21',
       '1994-09-18', '1979-01-20', '1985-10-20', '1972-11-27'],
      dtype=object)

In [13]:
print(abs['Social drinker'].unique())
print(abs['Social smoker'].unique())

['Y' 'Yes' 'No']
['No' 'Yes' 'Y']


In [14]:
abs['Weight'].unique()

array(['90', '98', '89', '68', '80', '65', '-', '88', '67', '69', '86',
       '84', '58', '83', '106', '95', '73', '70', '56', '63', '76', '75',
       '77', '79', '100', '108', '94'], dtype=object)

In [15]:
#ver a questao dos 0s (8h means all day probably)
abs['Absenteeism time in hours'].value_counts(dropna=False)


Absenteeism time in hours
8      216
2      170
3      124
1       97
4       65
0       48
16      21
24      18
5        7
32       7
40       7
80       5
64       3
120      3
112      3
7        2
56       2
104      1
48       1
Name: count, dtype: int64

In [17]:
abs['Days since previous absence'].unique()

array([ 0., nan])

statistics------------------------------------------------------------------------------------------

In [16]:
abs.describe().T

,count,mean,std,min,25%,50%,75%,max
Days since previous absence,781.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Transportation expense,800.0,221.92750,66.778732,118.0,179.0,225.0,260.0,388.0
Distance from Residence to Work,800.0,29.79875,14.875057,5.0,16.0,26.0,50.0,52.0
Estimated commute time,800.0,59.34875,31.301067,5.0,31.0,52.0,94.0,114.0
Years until retirement,800.0,26.74250,9.269498,-1.0,25.0,28.0,33.0,38.0
Education,800.0,1.28750,0.667383,1.0,1.0,1.0,1.0,4.0
Number of children,800.0,1.04375,1.120672,0.0,0.0,1.0,2.0,4.0
Number of pets,800.0,0.74500,1.323694,0.0,0.0,0.0,1.0,8.0
Height,800.0,172.09375,6.058122,163.0,169.0,170.0,172.0,196.0
Body mass index,800.0,26.64125,4.267354,19.0,24.0,25.0,31.0,38.0


In [ ]:
#statistics of categorical vars
data.describe(include = ['O']).T

,count,unique,top,freq
Reason for absence,740,28,Medical consultation,149
Month of absence,737,12,March,87
Day of the week,740,6,Monday,159
Seasons,573,4,Spring,155
Service time,800,19,18,150
Date of Birth,800,36,1987-04-08,122
Disciplinary failure,800,2,No,756
Social drinker,800,3,Yes,367
Social smoker,800,3,No,744
Weight,800,27,89,110


most frequent reason for absence is medical consultation

duplicates ----------------------------------

In [18]:
abs[abs.duplicated(keep=False)]

,Reason for absence,Month of absence,Day of the week,Seasons,Days since previous absence,Transportation expense,Distance from Residence to Work,Estimated commute time,Service time,Years until retirement,Date of Birth,Disciplinary failure,Education,Number of children,Social drinker,Social smoker,Number of pets,Weight,Height,Body mass index,Absenteeism time in hours
ID,,,,,,,,,,,,,,,,,,,,,
20,NaN,NaN,NaN,NaN,0.0,260,50,94,11,29,1989-02-20,No,1,4,Yes,No,0,65,168,23,8
20,NaN,NaN,NaN,NaN,0.0,260,50,94,11,29,1989-02-20,No,1,4,Yes,No,0,65,168,23,8
3,NaN,NaN,NaN,NaN,0.0,179,51,108,18,27,1987-04-08,No,1,0,Y,No,0,89,170,31,3
34,Medical consultation,October,Tuesday,Spring,0.0,118,10,22,10,28,1988-05-10,No,1,0,No,No,0,83,172,28,3
34,Medical consultation,October,Tuesday,Spring,0.0,118,10,22,10,28,1988-05-10,No,1,0,No,No,0,83,172,28,3
10,Patient follow-up,December,Wednesday,NaN,0.0,361,52,114,3,37,1997-07-08,No,1,1,Yes,No,4,80,172,27,8
28,Medical consultation,December,Tuesday,Spring,0.0,225,26,50,9,37,1997-08-03,No,1,1,No,No,2,69,169,24,2
10,Patient follow-up,December,Wednesday,NaN,0.0,361,52,114,3,37,1997-07-08,No,1,1,Yes,No,4,80,172,27,8
28,NaN,NaN,NaN,NaN,0.0,225,26,50,9,37,1997-08-03,No,1,1,No,No,2,69,169,24,1


damn